<a href="https://colab.research.google.com/github/talhaanwarch/mia-covid19/blob/main/ag2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install packages

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-4d7d703c-3458-861d-337f-5682100cabcc)


In [2]:
#restart kernel after installing packages and run it again
%%capture 
!pip install -U  pip
!pip install -U setuptools wheel
!pip install -U "mxnet_cu110<2.0.0"
!pip install  autogluon  # autogluon==0.2.0

# Get data from drive

In [3]:
#connect drive with colab notebook/
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
#copy data from drive to colab
!cp /content/drive/MyDrive/covid/train.zip /content/
!cp /content/drive/MyDrive/covid/val.zip /content/

In [5]:
#unzip data
%%capture 
!unzip /content/train.zip
!rm /content/train.zip
!mv /content/resized /content/train
#val data
!unzip /content/val.zip -d val
!rm /content/val.zip


# import packages

In [6]:
import pandas as pd
import numpy as np
from glob import glob
import cv2
import pandas as pd
import autogluon.core as ag
from autogluon.vision import ImagePredictor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


# prepare data

In [16]:
glob('/content/train'+'/*/')[1].split('/')[-2]

['non-covid', 'non-covid']

In [71]:
def flatten(path,label=None):
  '''
  path: images path in a list
  label: corresponding label in list
  '''
  path = [item for sublist in path for item in sublist]
  label = [item for sublist in label for item in sublist]
  return path, label
  

def prepare(path,split=False,flatten=True):
  '''
  Pass the path of main folder in which binary class folders are present
  Split True will split the data and return two sets
  '''
  main_path=glob(path+'/*/')
  pos_path=[glob(folder+'*.jpg') for folder in glob(main_path[0]+'*/')]
  neg_path=[glob(folder+'*.jpg') for folder in glob(main_path[1]+'*/')]
  #get folder name as label
  pos_label=[[main_path[0].split('/')[-2]]*len(i)  for i in pos_path]
  neg_label=[[main_path[1].split('/')[-2]]*len(i)  for i in neg_path]
  path=pos_path+neg_path
  label=pos_label+neg_label
  if flatten:
    if split:
      train_path,val_path,train_label,val_label=train_test_split(path,label,test_size=0.2)
      train_path,train_label=flatten(train_path,train_label)
      val_path,val_label=flatten(val_path,val_label)
      return train_path,val_path,train_label,val_label
    else:
      path,label=flatten(path,label)
      return path,label
  else:
    return path,label
  

In [44]:
train_path,val_path,train_label,val_label=prepare('/content/train',split=True)
test_path,test_label=prepare('/content/val',split=False)
print('Train Images and labels',len(train_path),len(train_label))
print('Val Images and labels',len(val_path),len(val_label))
print('Test Images and labels',len(test_path),len(test_label))

Train Images and labels 266634 266634
Val Images and labels 69038 69038
Test Images and labels 75532 75532


In [50]:
train_df=pd.DataFrame(zip(train_path,train_label),columns=['image','label'])
train_df = train_df.sample(frac=1).reset_index(drop=True)
train_df.label=train_df.label.map({'non-covid':0,'covid':1})
train_df.to_csv('/content/drive/MyDrive/covid/train_df.csv',index=False)
train_df.head()

,image,label
0,/content/train/non-covid/ct_scan_760/110.jpg,0
1,/content/train/non-covid/ct_scan_383/120.jpg,0
2,/content/train/non-covid/ct_scan_450/174.jpg,0
3,/content/train/covid/ct_scan_636/80.jpg,1
4,/content/train/non-covid/ct_scan_324/464.jpg,0


In [51]:
val_df=pd.DataFrame(zip(val_path,val_label),columns=['image','label'])
val_df = val_df.sample(frac=1).reset_index(drop=True)
val_df.label=val_df.label.map({'non-covid':0,'covid':1})
val_df.to_csv('/content/drive/MyDrive/covid/val_df.csv',index=False)
val_df.head()

,image,label
0,/content/train/covid/ct_scan_227/1.jpg,1
1,/content/train/covid/ct_scan_380/403.jpg,1
2,/content/train/non-covid/ct_scan_237/332.jpg,0
3,/content/train/covid/ct_scan_494/65.jpg,1
4,/content/train/non-covid/ct_scan_415/16.jpg,0


In [52]:
test_df=pd.DataFrame(zip(test_path,test_label),columns=['image','label'])
test_df = test_df.sample(frac=1).reset_index(drop=True)
test_df.label=test_df.label.map({'non-covid':0,'covid':1})
test_df.to_csv('/content/drive/MyDrive/covid/test_df.csv',index=False)
test_df.head()

,image,label
0,/content/val/covid/ct_scan_41/166.jpg,1
1,/content/val/covid/ct_scan_148/678.jpg,1
2,/content/val/covid/ct_scan_86/54.jpg,1
3,/content/val/covid/ct_scan_57/255.jpg,1
4,/content/val/non-covid/ct_scan_42/515.jpg,0


# train model

In [63]:
%%time
predictor = ImagePredictor(verbosity=1)
predictor.fit(train_df,tuning_data=val_df, hyperparameters={'epochs': 5}) 

The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1


RuntimeError: ignored

In [ ]:
fit_result = predictor.fit_summary()
print('Top-1 train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))
fit_result


In [56]:
val_df=ImagePredictor.Dataset.from_csv('/content/drive/MyDrive/covid/val_df.csv')
res = predictor.evaluate(val_df)
print('Top-1 test acc: %.3f' % res[0])
val_pred=predictor.predict(test_df)
print(classification_report(test_df.label,val_pred))

Top-1 test acc: 0.788
              precision    recall  f1-score   support

           0       0.78      0.88      0.83     40516
           1       0.84      0.71      0.77     35016

    accuracy                           0.80     75532
   macro avg       0.81      0.80      0.80     75532
weighted avg       0.81      0.80      0.80     75532



In [57]:
test_df=ImagePredictor.Dataset.from_csv('/content/drive/MyDrive/covid/test_df.csv')
res = predictor.evaluate(test_df)
print('Top-1 test acc: %.3f' % res[0])

Top-1 test acc: 0.804


In [62]:
test_pred=predictor.predict(test_df)
print(classification_report(test_df.label,test_pred))

AttributeError: ignored

In [61]:
predictor.save('/content/drive/MyDrive/covid/predictor.ag')


AttributeError: ignored

# final evaluation on validation data

In [64]:
predictor=predictor.load('/content/drive/MyDrive/covid/predictor.ag')


In [72]:
path,label=prepare('/content/val',split=False,flatten=False)
len(path),len(label)

(374, 374)

In [98]:
#now loop each folder of each directoy and create a dataframe for each folder
def evaluate(path,label):
  test_pred,test_true=[],[]
  for i, j in zip(path,label):
    test_df=pd.DataFrame(zip(i,j),columns=['image','label'])
    test_df.label=test_df.label.map({'non-covid':0,'covid':1})
    test_df.to_csv('test_df.csv',index=False)
    test_df=ImagePredictor.Dataset.from_csv('/content/test_df.csv')
    pred=list(predictor.predict(test_df))
    #find maximum occurent of element
    test_pred.append(max(pred,key=pred.count))
    test_true.append(test_df.label.mean())
  return test_pred,test_true



In [ ]:
test_pred,test_true=evaluate(path,label)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_true,test_pred))